In [39]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
import seaborn as sns
import math
import numpy as np
import pandas as pd
!pip install scikit-plot
import scikitplot
import seaborn as sns
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
import time

In [40]:
model_dir = "/content/drive/MyDrive/Kaggle/fer2013/model_1612972674.3130167.h5"
model = keras.models.load_model(model_dir)

In [41]:
emotion_label = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5:'surprise', 6:'neutral'}

# Test Using a Video

In [42]:
import cv2
import dlib
detector = dlib.get_frontal_face_detector()
def rect2bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

def getFaceBoundingBox(rgbImg):
        assert rgbImg is not None
        faces = detector(rgbImg, 1)
        if (len(faces) > 0) or len(faces) == 1:
            # return max(faces, key=lambda rect: rect.width() * rect.height())
            return faces
        else:
            return None

def img2csv(img):
    img = cv2.resize(img,(48,48))
    return img

from google.colab.patches import cv2_imshow

def img2data(img):
    result = []
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    boxes = getFaceBoundingBox(img)
    if boxes == None:
        return None
    for box in boxes:
        x, y, w, h = rect2bb(box)
        face = img[y: y+h, x: x+w]
        gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        data = img2csv(gray)
        result.append([data, gray])
    return result


In [48]:
test_vid_dir = "/content/drive/MyDrive/CUAI/2401-2800-수정본/clip_2502/clip_2502.mp4"
from google.colab.patches import cv2_imshow


In [49]:
vs = cv2.VideoCapture(test_vid_dir)

count = 0
while True:
    ret, img = vs.read()
    count += 1
    if ret and count % 100 == 0:
        cv2_imshow(img)
        results = img2data(img)
        for r in results:
            t = np.array(r)
            data = r[0]
            face = r[1]
            cv2_imshow(face)
            data = np.reshape(data, (1,48,48,1)) / 255.
            pred = model.predict(data)
            pred = np.argmax(pred)
            print(emotion_label[pred])
            keys = list(emotion_label.keys())
            perc = np.round(model.predict(data) * 100)
            for i in range(len(keys)):
                print(emotion_label[keys[i]], ":", perc[0][i])
    elif ret:
        pass
    else:
        break

Output hidden; open in https://colab.research.google.com to view.